In [25]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [26]:
df = pd.read_csv(Path('./data/20210613.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,08/06/2021 19:56:51,Metropolis (2001),Grave of the Fireflies (1988),Perfect Blue (1997),Redline (2009),Vampire Hunter D (1985),Mind Game (2004),"HONEY, I'M GAAAY",ICELAND,MURDER MYSTERY DINNER PARTY,ONE LAST JOB,[Default to Nicolas Cage],GUNS GUNS GUNS!,BUNGA BUNGA
1,08/06/2021 20:10:00,Mind Game (2004),Grave of the Fireflies (1988),Vampire Hunter D (1985),Metropolis (2001),NaN,NaN,"HONEY, I'M GAAAY",ONE LAST JOB,BUNGA BUNGA,GUNS GUNS GUNS!,[Default to Nicolas Cage],MURDER MYSTERY DINNER PARTY,ICELAND
2,08/06/2021 20:46:38,Vampire Hunter D (1985),Metropolis (2001),Perfect Blue (1997),Redline (2009),Mind Game (2004),Grave of the Fireflies (1988),BUNGA BUNGA,GUNS GUNS GUNS!,ONE LAST JOB,MURDER MYSTERY DINNER PARTY,"HONEY, I'M GAAAY",ICELAND,[Default to Nicolas Cage]
3,09/06/2021 05:35:49,Redline (2009),Mind Game (2004),Vampire Hunter D (1985),Metropolis (2001),Perfect Blue (1997),Grave of the Fireflies (1988),GUNS GUNS GUNS!,"HONEY, I'M GAAAY",ONE LAST JOB,BUNGA BUNGA,[Default to Nicolas Cage],MURDER MYSTERY DINNER PARTY,ICELAND
4,09/06/2021 06:06:15,Redline (2009),Grave of the Fireflies (1988),Vampire Hunter D (1985),Mind Game (2004),Perfect Blue (1997),NaN,MURDER MYSTERY DINNER PARTY,ONE LAST JOB,GUNS GUNS GUNS!,BUNGA BUNGA,[Default to Nicolas Cage],NaN,NaN
5,09/06/2021 06:06:56,Grave of the Fireflies (1988),Redline (2009),Perfect Blue (1997),Mind Game (2004),Metropolis (2001),Vampire Hunter D (1985),ONE LAST JOB,MURDER MYSTERY DINNER PARTY,BUNGA BUNGA,ICELAND,"HONEY, I'M GAAAY",GUNS GUNS GUNS!,[Default to Nicolas Cage]


In [27]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [28]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Metropolis (2001)', 'Grave of the Fireflies (1988)',
       'Perfect Blue (1997)', 'Redline (2009)', 'Vampire Hunter D (1985)',
       'Mind Game (2004)'], dtype=object)

In [29]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [30]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [31]:
submission_dict

{'Metropolis (2001)': <Candidate('Metropolis (2001)')>,
 'Grave of the Fireflies (1988)': <Candidate('Grave of the Fireflies (1988)')>,
 'Perfect Blue (1997)': <Candidate('Perfect Blue (1997)')>,
 'Redline (2009)': <Candidate('Redline (2009)')>,
 'Vampire Hunter D (1985)': <Candidate('Vampire Hunter D (1985)')>,
 'Mind Game (2004)': <Candidate('Mind Game (2004)')>}

In [32]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [33]:
ballots

[<Ballot(Metropolis (2001), Grave of the Fireflies (1988), Perfect Blue (1997), Redline (2009), Vampire Hunter D (1985), Mind Game (2004))>,
 <Ballot(Mind Game (2004), Grave of the Fireflies (1988), Vampire Hunter D (1985), Metropolis (2001))>,
 <Ballot(Vampire Hunter D (1985), Metropolis (2001), Perfect Blue (1997), Redline (2009), Mind Game (2004), Grave of the Fireflies (1988))>,
 <Ballot(Redline (2009), Mind Game (2004), Vampire Hunter D (1985), Metropolis (2001), Perfect Blue (1997), Grave of the Fireflies (1988))>,
 <Ballot(Redline (2009), Grave of the Fireflies (1988), Vampire Hunter D (1985), Mind Game (2004), Perfect Blue (1997))>,
 <Ballot(Grave of the Fireflies (1988), Redline (2009), Perfect Blue (1997), Mind Game (2004), Metropolis (2001), Vampire Hunter D (1985))>]

In [34]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [35]:
print(election_result)

ROUND 1
Candidate                        Votes  Status
-----------------------------  -------  --------
Redline (2009)                       2  Hopeful
Grave of the Fireflies (1988)        1  Hopeful
Mind Game (2004)                     1  Hopeful
Metropolis (2001)                    1  Hopeful
Vampire Hunter D (1985)              1  Rejected
Perfect Blue (1997)                  0  Rejected

ROUND 2
Candidate                        Votes  Status
-----------------------------  -------  --------
Redline (2009)                       2  Hopeful
Metropolis (2001)                    2  Hopeful
Grave of the Fireflies (1988)        1  Rejected
Mind Game (2004)                     1  Rejected
Vampire Hunter D (1985)              0  Rejected
Perfect Blue (1997)                  0  Rejected

FINAL RESULT
Candidate                        Votes  Status
-----------------------------  -------  --------
Metropolis (2001)                    3  Elected
Redline (2009)                       3  Rejected
Gr

In [36]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
HONEY, I'M GAAAY                   2  Hopeful
ONE LAST JOB                       1  Hopeful
MURDER MYSTERY DINNER PARTY        1  Hopeful
GUNS GUNS GUNS!                    1  Hopeful
BUNGA BUNGA                        1  Rejected
ICELAND                            0  Rejected
[Default to Nicolas Cage]          0  Rejected

ROUND 2
Candidate                      Votes  Status
---------------------------  -------  --------
HONEY, I'M GAAAY                   2  Hopeful
GUNS GUNS GUNS!                    2  Hopeful
ONE LAST JOB                       1  Rejected
MURDER MYSTERY DINNER PARTY        1  Rejected
BUNGA BUNGA                        0  Rejected
ICELAND                            0  Rejected
[Default to Nicolas Cage]          0  Rejected

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
GUNS GUNS GUNS!                    3  Ele